In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
class var:
    def __init__(self,data,weights):
        self.data=data
        self.weights=weights 
    
    def port_returns(self):
        import pandas as pd
        import numpy as np
        returns=pd.DataFrame()
        for f in self.data.columns:
            returns[f]=self.data[f].pct_change()
        returns=returns.dropna()
        total=[]
        for b in range(len(returns)-1):
            array_returns=np.transpose(np.array(returns.iloc[b,:]))
            array_weights=np.array(self.weights)
            total.append(np.dot(array_returns,array_weights))
        self.total_return=total
        return np.array(total)
    
    def parametric(self,conf_level): #Parametric VaR
        risk=np.std(self.total_return)
        meann=np.mean(self.total_return)
        import scipy.stats as stats 
        z=stats.norm.ppf(1-conf_level,0,1)
        var=(z*risk)+meann
        self.risk=risk
        self.meann=meann
        return var

    def historic(self,conf_level): #Historical VaR
        conteo=len(self.total_return)
        indice=round((1-conf_level)*conteo)
        port_ordenado=np.sort(self.total_return,None)
        var_historico=port_ordenado[indice-1]
        return var_historico
    
    def sim_historical(self,conf_level,n_simulations): #Historical simulation VaR
        simulaciones=np.random.randint(low=1,high=len(self.total_return),size=(n_simulations,1))
        indices_sim=[]
        retornos_sim_historical=np.array(self.total_return)
        for y in range(len(simulaciones)-1):
            valor=retornos_sim_historical[simulaciones[y]-1]
            indices_sim.append(valor)
        arreglo_indices=pd.DataFrame(np.reshape(np.array(indices_sim),newshape=(n_simulations-1,1)))
        arreglo_indices.columns=["Historical Simulation VaR"]
        var_sim_historico=arreglo_indices.quantile(1-conf_level)
        return var_sim_historico
    
    def monte_carlo(self,conf_level,n_simulations):#Monte Carlo VaR
        import scipy.stats as stats
        import random
        aleatorio=[]
        for j in range(n_simulations):
            aleatorio.append(random.random())
        z_mc=[]
        for t in range(len(aleatorio)-1):
            z_mc.append(stats.norm.ppf(aleatorio[t],0,1))
        z_sim_mc=np.array(z_mc)
        retornos_sim=pd.DataFrame((z_sim_mc*self.risk)+self.meann)
        retornos_sim.columns=["Monte Carlo VaR"]
        var_mont_car=retornos_sim.quantile(1-conf_level)
        return var_mont_car

Import equity data and weights

In [23]:
data_2=pd.read_excel("Proyecto var.xlsx",index_col=0)
weights=pd.read_excel("Proyecto var.xlsx",sheet_name="pesos")

In [30]:
data_2.head()

,COCA COLA,EXXON MOBIL,JOHNSON & JOHNSON,JP MORGAN,APPLE
Fecha,,,,,
2018-01-02,39.689281,66.545746,124.394119,95.053246,41.075405
2018-01-03,39.602119,67.852707,125.582382,95.150124,41.068253
2018-01-04,40.159908,67.946625,125.573448,96.513199,41.259010
2018-01-05,40.151192,67.891838,126.609863,95.893608,41.728756
2018-01-08,40.090179,68.197075,126.770683,96.035217,41.573765


In [33]:
weights.index=data_2.columns
weights

,Pesos
COCA COLA,0.15
EXXON MOBIL,0.21
JOHNSON & JOHNSON,0.18
JP MORGAN,0.14
APPLE,0.32


Creating object of the class

In [24]:
object_1=var(data=data_2,weights=weights)

Calculating the parametric Value At Risk

In [25]:
portafolio=object_1.port_returns() # Portfolio returns
object_1.parametric(conf_level=0.95) 

-0.022327111549003655

Calculating the historic Value At Risk

In [26]:
object_1.historic(0.95)

-0.020497663846085652

Calculating the Historical Simulation Value At Risk

In [28]:
object_1.sim_historical(0.95, 500)

Historical Simulation VaR   -0.029473
Name: 0.050000000000000044, dtype: float64

Calculating the Monte Carlo Value At Risk

In [29]:
object_1.monte_carlo(0.95, 500) 

Monte Carlo VaR   -0.024533
Name: 0.050000000000000044, dtype: float64